In [1]:
1+1

2

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
from dotenv import load_dotenv

load_dotenv()

# os.environ["OPENAI_API_KEY"] = ""
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. 임베딩 모델 초기화
# OpenAI의 text-embedding-ada-002 모델 사용
# 기본 1536차원 : 1536 × 4바이트 = 6.1KB per 문서
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 1024차원으로 축소: 512 × 4바이트 = 2.0KB per 문서 (약 67% 절약)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [4]:
import os
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Tesseract 실행 파일 경로를 직접 지정
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


def load_and_process_document_with_ocr(file_path: str):
    """
    PDF 문서에서 텍스트를 추출합니다.
    텍스트가 없는 페이지는 이미지로 변환 후 OCR 수행.
    추출된 텍스트를 LangChain Document로 변환 후 청크 분할하여 반환.
    """
    ext = os.path.splitext(file_path)[1].lower()

    if ext != ".pdf":
        raise ValueError("현재는 PDF 파일만 지원합니다.")

    raw_docs = []
    pdf = fitz.open(file_path)

    for page_idx in range(len(pdf)):
        page = pdf.load_page(page_idx)
        text = page.get_text("text")

        if not text.strip():
            # 텍스트가 없으면 이미지로 변환 후 OCR 수행
            print(f"페이지 {page_idx + 1}: 텍스트가 없어 OCR 수행 중...")
            pix = page.get_pixmap()
            img_bytes = pix.tobytes("png")
            img = Image.open(io.BytesIO(img_bytes))
            ocr_text = pytesseract.image_to_string(img, lang="kor+eng")
            text = ocr_text

        if text.strip():
            raw_docs.append(
                Document(
                    page_content=text,
                    metadata={"source": file_path, "page": page_idx + 1},
                )
            )
        else:
            print(f"페이지 {page_idx + 1}: OCR 후에도 텍스트를 찾을 수 없습니다.")

    pdf.close()

    # 텍스트 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""],
    )

    chunks = splitter.split_documents(raw_docs)
    return chunks

In [5]:
chunks = load_and_process_document_with_ocr("sample_02.pdf")
print(f"청크 개수: {len(chunks)}")

페이지 1: 텍스트가 없어 OCR 수행 중...
페이지 2: 텍스트가 없어 OCR 수행 중...
페이지 3: 텍스트가 없어 OCR 수행 중...
페이지 4: 텍스트가 없어 OCR 수행 중...
페이지 5: 텍스트가 없어 OCR 수행 중...
청크 개수: 5


In [6]:
vectorstore = Chroma.from_documents(
    documents=chunks, embedding=embeddings, persist_directory="./chroma_db"
)

type(vectorstore)

langchain_community.vectorstores.chroma.Chroma

In [7]:
# 1. 기본 유사도 검색
def search_similar_documents(query, k=3):
    """
    쿼리와 유사한 문서를 검색하는 함수

    Args:
        query (str): 검색할 쿼리
        k (int): 반환할 문서 수

    Returns:
        list: 유사한 문서 리스트
    """
    # 유사도 검색 수행
    similar_docs = vectorstore.similarity_search(query=query, k=k)  # 상위 k개 문서 반환

    return similar_docs


# 검색 실행
query = "청년일자리가 뭔가요?"
results = search_similar_documents(query)

print(f"검색 쿼리: {query}")
print("=" * 50)

for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")
    print("-" * 30)

검색 쿼리: 청년일자리가 뭔가요?
1. 정책 기본 정보

정책명

정책소개

정책 분야

지원 내용

사업 운영 기간
사업 신청 기간
지원 규모(명)

비고

청년정책 정보

첨년일자리 도약장려금

기업의 청년고용 SHS 지원하고, 취업애로 청년의 취업을 촉진함으로써,
ADS 활성화를 목적으로 하는 정책

0 (유형 1) 5인 이상 중소기업에서 취업애로청년을 정규직으로 채용하고
oR 이상 고용유지시 기업지원금 지급

ㅇ (유형) 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월
이상 고용유지시 기업지원금 지급하고, 해당 기업에서 18개월 이상 재직한
첨년에게 청년장기근속인센티브 지급

일자리

© (유형 1) 5인 이상 우선지원대상기업에서 취업애로청년을 정규직으로 채

용시 1년간 최대 720만원 지원<6[> *취업애로정년: 만 15-34세의 4개월
이상 실업, 고졸 이하 정년 등 <br> ㅇ(유형2) 5인 이상의 제조업 등 빈일자

리 업종 중소기업에서 BAS 정규직으로 채용시 1년간 최대 720만원 지원

및 해당 기업에 취업 후 18개월 이상 재직한 청년 근로자에게 최대 480만원

지원(18.24개월차 각 240만원)

2025년 1월 1일 ~ 2025년 128 31일
2025년 1월 1일 ~ 2025년 12월 31일<61/>

제한없음

1페이지
------------------------------
2. 를 신청 자격

만 15세-만 34세

[거주지]
전국

[25]
,기업)매줄액 기준 (정년)해당없음

제한없음

제한없음

재직자

제한없음

2페이지
------------------------------
3. mle 자격

추가 단서 사항

참여 제한 대상

@ 근로조건<61>- 정규직 채용 후 6개월 이상 DBRA <br>* 계약직 채용
시 채용일로부터 3개월 이내 정규직 전환<6(>- 주 30시간 이상 근로 및 최
저임금 이상 지급<6「>- 고용보험 가입 필수<6「>(2 채용요건<6「>- '25.1.1.
이후 채용된 청년<6(>-

In [8]:
# 유사도 점수와 함께 검색
def search_with_scores(query, k=3):
    """
    유사도 점수와 함께 문서를 검색하는 함수
    """
    # 점수와 함께 검색
    results_with_scores = vectorstore.similarity_search_with_score(query=query, k=k)

    return results_with_scores


# 점수 기반 검색 실행
query = "청년정책이 뭔가요?"
scored_results = search_with_scores(query)

print(f"검색 쿼리: {query}")
print("=" * 50)

for i, (doc, score) in enumerate(scored_results, 1):
    print(f"{i}. 유사도 점수: {score:.4f}")
    print(f"   내용: {doc.page_content}")
    print("-" * 30)

검색 쿼리: 청년정책이 뭔가요?
1. 유사도 점수: 1.3837
   내용: 정책 기본 정보

정책명

정책소개

정책 분야

지원 내용

사업 운영 기간
사업 신청 기간
지원 규모(명)

비고

청년정책 정보

첨년일자리 도약장려금

기업의 청년고용 SHS 지원하고, 취업애로 청년의 취업을 촉진함으로써,
ADS 활성화를 목적으로 하는 정책

0 (유형 1) 5인 이상 중소기업에서 취업애로청년을 정규직으로 채용하고
oR 이상 고용유지시 기업지원금 지급

ㅇ (유형) 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월
이상 고용유지시 기업지원금 지급하고, 해당 기업에서 18개월 이상 재직한
첨년에게 청년장기근속인센티브 지급

일자리

© (유형 1) 5인 이상 우선지원대상기업에서 취업애로청년을 정규직으로 채

용시 1년간 최대 720만원 지원<6[> *취업애로정년: 만 15-34세의 4개월
이상 실업, 고졸 이하 정년 등 <br> ㅇ(유형2) 5인 이상의 제조업 등 빈일자

리 업종 중소기업에서 BAS 정규직으로 채용시 1년간 최대 720만원 지원

및 해당 기업에 취업 후 18개월 이상 재직한 청년 근로자에게 최대 480만원

지원(18.24개월차 각 240만원)

2025년 1월 1일 ~ 2025년 128 31일
2025년 1월 1일 ~ 2025년 12월 31일<61/>

제한없음

1페이지
------------------------------
2. 유사도 점수: 1.5888
   내용: 를 신청 자격

만 15세-만 34세

[거주지]
전국

[25]
,기업)매줄액 기준 (정년)해당없음

제한없음

제한없음

재직자

제한없음

2페이지
------------------------------
3. 유사도 점수: 1.6035
   내용: engage

신청 절차

심사 및 발표
신청 사이트
제출 서류

신청방법 및 지원절차<6[>(참여신청) 기업이 운영기관에 채용계획 MBA
산시스템 활용)-> 운명기관의 검토 및 승인-> 운영기관-기업 간 지원협약